<a href="https://colab.research.google.com/github/pachterlab/scATAK/blob/main/scATAK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!curl -Ls https://github.com/pachterlab/scATAK/blob/main/data/min-R1.fastq.gz?raw=true > min-R1.fastq.gz
!curl -Ls https://github.com/pachterlab/scATAK/blob/main/data/min-R2.fastq.gz?raw=true > min-R2.fastq.gz
!curl -Ls https://github.com/pachterlab/scATAK/blob/main/data/min-R3.fastq.gz?raw=true > min-R3.fastq.gz

In [4]:
# install minimap for mapping paired end reads
!wget --quiet --show-progress "https://github.com/lh3/minimap2/releases/download/v2.24/minimap2-2.24_x64-linux.tar.bz2"
!tar -jxvf minimap2-2.24_x64-linux.tar.bz2
!cp minimap2-2.24_x64-linux/minimap2 /usr/local/bin

# Insstall sambamba for sam file manipulation
!wget --quiet --show-progress https://github.com/biod/sambamba/releases/download/v0.8.2/sambamba-0.8.2-linux-amd64-static.gz
!gunzip sambamba-0.8.2-linux-amd64-static.gz
!chmod +x /content/sambamba-0.8.2-linux-amd64-static
!mv sambamba-0.8.2-linux-amd64-static /usr/local/bin/sambamba

# Genrich for peak callling
!git clone https://github.com/jsh58/Genrich.git
!cd Genrich && make && cp Genrich /usr/local/bin

# bedtools for slicing the peaks
!wget --quiet --show-progress https://github.com/arq5x/bedtools2/releases/download/v2.30.0/bedtools.static.binary
!chmod +x /content/bedtools.static.binary
!mv /content/bedtools.static.binary /usr/local/bin/bedtools

minimap2-2.24_x64-l 100%[===================>]   2.04M  --.-KB/s    in 0.06s   
minimap2-2.24_x64-linux/
minimap2-2.24_x64-linux/LICENSE.txt
minimap2-2.24_x64-linux/k8
minimap2-2.24_x64-linux/README-js.md
minimap2-2.24_x64-linux/paftools.js
minimap2-2.24_x64-linux/minimap2
minimap2-2.24_x64-linux/README.md
minimap2-2.24_x64-linux/cookbook.md
minimap2-2.24_x64-linux/minimap2.1
minimap2-2.24_x64-linux/NEWS.md


In [8]:
!minimap2

Usage: minimap2 [options] <target.fa>|<target.idx> [query.fa] [...]
Options:
  Indexing:
    -H           use homopolymer-compressed k-mer (preferrable for PacBio)
    -k INT       k-mer size (no larger than 28) [15]
    -w INT       minimizer window size [10]
    -I NUM       split index for every ~NUM input bases [4G]
    -d FILE      dump index to FILE []
  Mapping:
    -f FLOAT     filter out top FLOAT fraction of repetitive minimizers [0.0002]
    -g NUM       stop chain enlongation if there are no minimizers in INT-bp [5000]
    -G NUM       max intron length (effective with -xsplice; changing -r) [200k]
    -F NUM       max fragment length (effective with -xsr or in the fragment mode) [800]
    -r NUM[,NUM] chaining/alignment bandwidth and long-join bandwidth [500,20000]
    -n INT       minimal number of minimizers on a chain [3]
    -m INT       minimal chaining score (matching bases minus log gap penalty) [40]
    -X           skip self and dual mappings (for the all-vs-all m

In [22]:
!sambamba


sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.27.1 / DMD v2.097.2 / LLVM11.0.0 / bootstrap LDC - the LLVM D compiler (1.27.1)


Usage: sambamba [command] [args...]

  Available commands:

    view        view contents and convert from one format
                to another (SAM/BAM/JSON/UNPACK)
    index       build index (BAI)
    merge       merge files (BAM)
    sort        sort file (BAM)
    slice       slice file (BAM using BED)
    markdup     mark or remove duplicates (BAM)
    subsample   subsample (BAM)
    flagstat    output statistics (BAM)
    depth       output statistics (BAM)
    validate    simple validator (BAM)

  No longer recommended:

    mpileup     parallel execution of samtools (BAM)

To get help on a particular command, call it without args.

Global options

    -q          quiet mode (do not show banner)

For bug reports and feature requests see

       https://github.com/biod/



In [32]:
!Genrich

Error! Need input/output files
Usage: ./Genrich  -t <file>  -o <file>  [optional arguments]
Required arguments:
  -t  <file>       Input SAM/BAM file(s) for experimental sample(s)
  -o  <file>       Output peak file (in ENCODE narrowPeak format)
Optional I/O arguments:
  -c  <file>       Input SAM/BAM file(s) for control sample(s)
  -f  <file>       Output bedgraph-ish file for p/q values
  -k  <file>       Output bedgraph-ish file for pileups and p-values
  -b  <file>       Output BED file for reads/fragments/intervals
  -R  <file>       Output file for PCR duplicates (only with -r)
Filtering options:
  -r               Remove PCR duplicates
  -e  <arg>        Comma-separated list of chromosomes to exclude
  -E  <file>       Input BED file(s) of genomic regions to exclude
  -m  <int>        Minimum MAPQ to keep an alignment (def. 0)
  -s  <float>      Keep sec alns with AS >= bestAS - <float> (def. 0)
  -y               Keep unpaired alignments (def. false)
  -w  <int>        Keep unp

In [43]:
!bedtools

bedtools is a powerful toolset for genome arithmetic.

Version:   v2.30.0
About:     developed in the quinlanlab.org and by many contributors worldwide.
Docs:      http://bedtools.readthedocs.io/
Code:      https://github.com/arq5x/bedtools2
Mail:      https://groups.google.com/forum/#!forum/bedtools-discuss

Usage:     bedtools <subcommand> [options]

The bedtools sub-commands include:

[ Genome arithmetic ]
    intersect     Find overlapping intervals in various ways.
    window        Find overlapping intervals within a window around an interval.
    closest       Find the closest, potentially non-overlapping interval.
    coverage      Compute the coverage over defined intervals.
    map           Apply a function to a column for each overlapping interval.
    genomecov     Compute the coverage over an entire genome.
    merge         Combine overlapping/nearby intervals into a single interval.
    cluster       Cluster (but don't merge) overlapping/nearby intervals.
    complement

In [9]:
!wget --quiet --show-progress "http://ftp.ensembl.org/pub/release-104/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz"

--2022-09-30 17:06:13--  http://ftp.ensembl.org/pub/release-104/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.139
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 881211416 (840M) [application/x-gzip]
Saving to: ‘Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz’

Homo_sapiens.GRCh38 100%[===================>] 840.39M  23.9MB/s    in 4m 24s  

2022-09-30 17:10:37 (3.18 MB/s) - ‘Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz’ saved [881211416/881211416]



In [10]:
!time minimap2 \
-d ref.mmi \
Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

tcmalloc: large alloc 1073741824 bytes == 0x6a698000 @  0x7effc6ef72a4 0x414d2d 0x406c48 0x4129cf 0x7effc66f06db 0x7effc641961f
tcmalloc: large alloc 2147483648 bytes == 0x27d99e000 @  0x7effc6ef72a4 0x406fee 0x4129cf 0x7effc66f06db 0x7effc641961f
[M::mm_idx_gen::127.246*1.68] collected minimizers
[M::mm_idx_gen::161.853*1.73] sorted minimizers
[M::main::190.503*1.54] loaded/built the index for 194 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 194
[M::mm_idx_stat::192.955*1.53] distinct minimizers: 100159079 (38.79% are singletons); average occurrences: 5.540; average spacing: 5.586; total length: 3099750718
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -d ref.mmi Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
[M::main] Real time: 194.609 sec; CPU: 296.872 sec; Peak RSS: 11.262 GB

real	3m15.225s
user	4m14.967s
sys	0m42.510s


In [13]:
!time minimap2 -ax sr -t2 ref.mmi \
   <(paste \
     <(zcat min-R1.fastq.gz) \
     <(zcat min-R2.fastq.gz) \
     | awk -v blen="16" '{if(NR%4==1) header=$1; if(NR%4==2) print header"_"substr($2, 0, blen)"_\n"$1; if(NR%4==3 || NR%4==0) print $1;}') \
   <(paste \
     <(zcat min-R3.fastq.gz) \
     <(zcat min-R2.fastq.gz) \
     | awk -v blen="16" '{if(NR%4==1) header=$1; if(NR%4==2) print header"_"substr($2, 0, blen)"_\n"$1; if(NR%4==3 || NR%4==0) print $1;}') \
   > peaks.sam

tcmalloc: large alloc 1549877248 bytes == 0x17ea7a000 @  0x7f145e3421e7 0x40a494 0x40a9b8 0x402995 0x7f145d765c87 0x4018b9
[WARNING] Indexing parameters (-k, -w or -H) overridden by parameters used in the prebuilt index.
[M::main::56.680*0.37] loaded/built the index for 194 target sequence(s)
[M::mm_mapopt_update::56.680*0.37] mid_occ = 1000
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 194
[M::mm_idx_stat::57.890*0.38] distinct minimizers: 100159079 (38.79% are singletons); average occurrences: 5.540; average spacing: 5.586; total length: 3099750718
[M::worker_pipeline::103.114*0.99] mapped 500000 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -ax sr -t2 ref.mmi /dev/fd/63 /dev/fd/62
[M::main] Real time: 104.181 sec; CPU: 103.127 sec; Peak RSS: 7.617 GB

real	1m44.725s
user	1m28.124s
sys	0m18.010s


In [23]:
!sambamba view -t 2 -f bam -S -o peaks.bam peaks.sam
!rm peaks.sam


sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.27.1 / DMD v2.097.2 / LLVM11.0.0 / bootstrap LDC - the LLVM D compiler (1.27.1)



In [25]:
!time sambamba sort -n -t 2 -m 8GB --tmpdir=./ peaks.bam


sambamba 0.8.2
 by Artem Tarasov and Pjotr Prins (C) 2012-2021
    LDC 1.27.1 / DMD v2.097.2 / LLVM11.0.0 / bootstrap LDC - the LLVM D compiler (1.27.1)


real	0m3.770s
user	0m6.507s
sys	0m0.481s


In [34]:
!time Genrich -t peaks.sorted.bam -o peaks.narrowPeak -f peaks.log -v

Processing experimental file #0: peaks.sorted.bam
  BAM records analyzed:      500166
    Unmapped:                 25581
    Supp./dups/lowQual:         166
    Paired alignments:       447022
    Unpaired alignments:      27397
  Fragments analyzed:        223511
    Full fragments:          223511
      (avg. length: 174.4bp)
- control file #0 not provided -
  Background pileup value: 0.012576
Peak-calling parameters:
  Genome length: 3099750718bp
  Significance threshold: -log(p) > 2.000
  Min. AUC: 200.000
  Max. gap between sites: 100bp
Peaks identified: 150592 (32552110bp)

real	0m21.208s
user	0m14.941s
sys	0m6.146s


In [35]:
!head peaks.narrowPeak

1	10015	10460	peak_0	1000	.	3304.893066	12.412954	-1	105
1	11154	11266	peak_1	1000	.	449.575562	6.782719	-1	87
1	29210	29373	peak_2	1000	.	779.583130	6.782719	-1	81
1	29480	29798	peak_3	1000	.	1573.081665	8.581930	-1	150
1	99502	99868	peak_4	1000	.	1750.474976	6.782719	-1	183
1	104185	104255	peak_5	1000	.	334.790314	6.782719	-1	35
1	143396	143517	peak_6	1000	.	578.708984	6.782719	-1	60
1	149332	149519	peak_7	1000	.	894.368408	6.782719	-1	93
1	180546	180929	peak_8	1000	.	1831.781250	6.782719	-1	191
1	181290	181834	peak_9	1000	.	3828.631592	11.901150	-1	194


In [45]:
!cat peaks.narrowPeak | bedtools sort | bedtools merge > atac_all_peaks.bed

In [47]:
!bedtools getfasta -fi Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz -bed atac_all_peaks.bed -fo atac_all_peaks.fa

index file Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz.fai not found, generating...
ERROR: mismatched line lengths at line 3 within sequence 
File not suitable for fasta index generation.
